In [1]:
import os, time
import cv2
import xml.etree.ElementTree as ET
from PIL import Image
import numpy as np
import random
from pathlib import Path

In [2]:
SEED_XML_DIR = '/data/darknet/python/xml/'
SEED_IMG_DIR = '/data/darknet/python/image/'
SEED_BKG_DIR = '/data/darknet/python/background/'
GENE_IMG_DIR = '../generated/images/jpgs/'
GENE_XML_DIR = '../generated/xml/'
GENE_OBJ_DIR = '../generated/images/'

In [3]:
# RT:RightTop 
# LB:LeftBottom 
# bbox: [xmin, xax, ymin, ymax]
def IOU(bbox_a, bbox_b):
    '''
    W = min(A.RT.x, B.RT.x) - max(A.LB.x, B.LB.x) 
    H = min(A.RT.y, B.RT.y) - max(A.LB.y, B.LB.y) 
    if W <= 0 or H <= 0: 
        return 0 
    SA = (A.RT.x - A.LB.x) * (A.RT.y - A.LB.y) 
    SB = (B.RT.x - B.LB.x) * (B.RT.y - B.LB.y) 
    cross = W * H return cross/(SA + SB - cross)
    '''
    W = min(bbox_a[1], bbox_b[1]) - max(bbox_a[0], bbox_b[0]) 
    H = min(bbox_a[3], bbox_b[3]) - max(bbox_a[2], bbox_b[2]) 
    if W <= 0 or H <= 0: 
        return 0
    SA = (bbox_a[1] - bbox_a[0]) * (bbox_a[3] - bbox_a[2]) 
    SB = (bbox_b[1] - bbox_b[0]) * (bbox_b[3] - bbox_b[2])  
    cross = W * H 
    return cross/(SA + SB - cross)

IOU([1,2,3,4], [1,2,3,3.5])


0.5

In [4]:
def blur_background_from_image_file(file, bboxes, extend_spaces=0):
    
    img = cv2.imread(file)
    #print(img.shape)
    #print(img)
    img_objs = []
    for bbox in bboxes:
        img_obj = img[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])]
        img_objs.append(img_obj)
    img = cv2.blur(img, (int(img.shape[0]/20), int(img.shape[1]/30)))
    #img = cv2.blur(img, (32, 32))
    img = cv2.blur(img, (int(img.shape[0]/80), int(img.shape[1]/120)))
    #img = cv2.blur(img, (8, 8))
    i = 0
    for bbox in bboxes:
        img[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])] = img_objs[i]
        i = i+1
    cv2.imwrite(GENE_IMG_DIR+file.split('/')[-1], img)
    return 
def get_obj_from_xml(xml):
    in_file = open(xml_name)
    tree=ET.parse(in_file)
    root = tree.getroot()
    return [obj for obj in root.iter('object')]

In [5]:
def past_to_background_from_image_file(file, bboxes, background_img_array, extend_spaces=0):
    
    img = cv2.imread(file)
    if(img.shape != background_img_array.shape):
        print('shape not match')
        return
    #print(img.shape)
    #print(img)
    img_objs = []
    for bbox in bboxes:
        img_obj = img[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])]
        img_objs.append(img_obj)
    i = 0
    for bbox in bboxes:
        background_img_array[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])] = img_objs[i]
        i = i+1
    cv2.imwrite(GENE_IMG_DIR+file.split('/')[-1], background_img_array)
    return 
def get_obj_from_xml(xml):
    in_file = open(xml_name)
    tree=ET.parse(in_file)
    root = tree.getroot()
    return [obj for obj in root.iter('object')]

In [9]:
seed_xml_names = os.listdir(SEED_XML_DIR)
seed_bkg_names = os.listdir(SEED_BKG_DIR)
bkg_heads = [bkg[:8] for bkg in seed_bkg_names]
print(bkg_heads)

seed_xml_names.sort()
#print(seed_xml_names)
#seed_img_names = os.listdir(SEED_IMG_DIR)
#seed_chg_names = os.listdir(SEED_CHG_DIR)

classes_ = ['bus', 'car', 'truck', 'motorbike', 'bicycle', 'person']
temp_index_array = [200, 201, 202, 203, 204, 205, 206, 207, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219]
#temp_index_array = [204]
last_frame_bboxes = []
for xml_name in seed_xml_names:
    #print(xml_name)   
    head = xml_name[:8]
    if(not (int(head) in temp_index_array)):
        continue
    index = bkg_heads.index(head)
    background_img_array = cv2.imread(SEED_BKG_DIR + seed_bkg_names[index])    
    
    '''
    head = xml_name[:11]
    if(not (int(head) > 214000 and int(head) < 215000)):
        continue
    '''

    if(not xml_name.endswith('.xml')):
        continue
    in_file = open(SEED_XML_DIR + xml_name)
    tree=ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    
    #w = int(size.find('width').text)    
    #h = int(size.find('height').text)
    this_frame_bboxes = []
    boxes = []
    objs = []
    b = []
    #wrong_detection_objs = [[130, 311, 226, 475]]
    #objs_iter = root.iter('object')
    N = 0
    array_object = root.findall('object')
    for obj in array_object:
        erro_obj_removed = False
        stable_bbox_removed = False
        difficult = obj.find('difficult').text
        cls_ = obj.find('name').text
        if cls_ not in classes_ or int(difficult)==1:
            continue
        xmlbox = obj.find('bndbox')
        b = [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
        
        '''
        for erro_bbox in wrong_detection_objs:
            if(IOU(erro_bbox, b) > 0.8):
                print(b)
                root.remove(obj)
                erro_obj_removed = True
                break
        if(erro_obj_removed):
            continue
        '''
        this_frame_bboxes.append(b)
        if(len(last_frame_bboxes)>0):
            for stable_bbox in last_frame_bboxes:
                if(IOU(stable_bbox, b) > 0.85):
                    #print(b)
                    root.remove(obj)
                    stable_bbox_removed = True
                    N = N+1
                    break
            if(stable_bbox_removed):
                continue
        #filter size which smaller than 16X16 fit for SSD300
        bbox_width = b[1] - b[0]
        bbox_heigt = b[3] - b[2]
        if(not(bbox_width > 16 or bbox_heigt > 16)):
            print('remove:{0}:{1}'.format(bbox_width, bbox_heigt))
            root.remove(obj)
            continue 
        else:
            print('add:{0}:{1}'.format(bbox_width, bbox_heigt))
        boxes.append(b)
        objs += obj
    last_frame_bboxes = this_frame_bboxes.copy()  
    #print(boxes)
    ###blur_background_from_image_file(SEED_IMG_DIR + xml_name[:-3] + 'jpg', boxes)
    if(len(boxes) == 0):
        continue
    past_to_background_from_image_file(SEED_IMG_DIR + xml_name[:-3] + 'jpg', boxes, background_img_array)
    tree.write(GENE_XML_DIR + xml_name)
    #break

['00000219', '00000204', '00000201', '00000211', '00000214', '00000200', '00000210', '00000203', '00000218', '00000213', '00000216', '00000205', '00000207', '00000202', '00000206', '00000215', '00000212', '00000217']
add:102.0:253.0
add:158.0:394.0
add:89.0:187.0
add:113.0:150.0
add:100.0:182.0
add:95.0:253.0
add:192.0:412.0
add:103.0:179.0
add:87.0:293.0
add:135.0:150.0
add:243.0:416.0
add:88.0:265.0
add:110.0:188.0
add:134.0:148.0
add:109.0:151.0
add:88.0:142.0
add:78.0:198.0
add:241.0:440.0
add:91.0:252.0
add:127.0:202.0
add:107.0:152.0
add:70.0:355.0
add:85.0:143.0
add:237.0:459.0
add:95.0:255.0
add:112.0:191.0
add:107.0:159.0
add:61.0:326.0
add:105.0:255.0
add:101.0:185.0
add:81.0:355.0
add:102.0:149.0
add:103.0:144.0
add:103.0:144.0
add:243.0:463.0
add:99.0:191.0
add:91.0:316.0
add:93.0:158.0
add:91.0:132.0
add:99.0:146.0
add:93.0:262.0
add:100.0:192.0
add:90.0:144.0
add:61.0:148.0
add:87.0:142.0
add:81.0:269.0
add:106.0:214.0
add:80.0:145.0
add:95.0:141.0
add:306.0:30.0
add:88.0

add:224.0:308.0
add:100.0:167.0
add:143.0:164.0
add:111.0:195.0
add:176.0:167.0
add:265.0:275.0
add:94.0:193.0
add:257.0:246.0
add:154.0:160.0
add:93.0:199.0
add:229.0:208.0
add:172.0:156.0
add:111.0:201.0
add:223.0:173.0
add:161.0:151.0
add:187.0:125.0
add:169.0:179.0
add:101.0:168.0
add:116.0:208.0
add:183.0:176.0
add:179.0:85.0
add:98.0:215.0
add:165.0:181.0
add:177.0:189.0
add:102.0:206.0
add:124.0:209.0
add:184.0:188.0
add:107.0:228.0
add:176.0:199.0
add:101.0:223.0
add:193.0:190.0
add:119.0:223.0
add:196.0:204.0
add:284.0:34.0
add:115.0:206.0
add:186.0:203.0
add:298.0:43.0
add:115.0:226.0
add:193.0:197.0
add:336.0:53.0
add:119.0:235.0
add:340.0:58.0
add:211.0:212.0
add:127.0:246.0
add:200.0:203.0
add:351.0:75.0
add:119.0:248.0
add:183.0:279.0
add:343.0:84.0
add:115.0:241.0
add:200.0:254.0
add:359.0:98.0
add:373.0:103.0
add:121.0:241.0
add:215.0:274.0
add:381.0:111.0
add:133.0:252.0
add:202.0:305.0
add:146.0:266.0
add:421.0:121.0
add:217.0:281.0
add:397.0:155.0
add:148.0:295.0
add

add:911.0:501.0
add:857.0:533.0
add:83.0:68.0
add:825.0:479.0
add:90.0:84.0
add:698.0:457.0
add:601.0:448.0
add:60.0:128.0
add:86.0:79.0
add:494.0:461.0
add:62.0:76.0
add:397.0:398.0
add:298.0:438.0
add:205.0:444.0
add:96.0:426.0
add:123.0:34.0
add:97.0:82.0
add:94.0:92.0
add:99.0:98.0
add:108.0:91.0
add:61.0:96.0
add:111.0:103.0
add:127.0:43.0
add:63.0:100.0
add:59.0:128.0
add:102.0:132.0
add:115.0:45.0
add:132.0:49.0
add:107.0:132.0
add:141.0:50.0
add:145.0:126.0
add:56.0:107.0
add:149.0:48.0
add:86.0:136.0
add:159.0:128.0
add:140.0:51.0
add:54.0:136.0
add:154.0:122.0
add:133.0:58.0
add:95.0:146.0
add:92.0:151.0
add:96.0:51.0
add:157.0:118.0
add:93.0:148.0
add:177.0:128.0
add:84.0:52.0
add:172.0:123.0
add:86.0:158.0
add:60.0:59.0
add:96.0:143.0
add:99.0:154.0
add:178.0:119.0
add:61.0:128.0
add:174.0:135.0
add:86.0:145.0
add:192.0:133.0
add:93.0:166.0
add:77.0:171.0
add:76.0:177.0
add:192.0:135.0
add:201.0:156.0
add:88.0:194.0
add:93.0:166.0
add:166.0:53.0
add:89.0:172.0
add:97.0:178.

add:161.0:276.0
add:494.0:356.0
add:236.0:210.0
add:59.0:127.0
add:490.0:348.0
add:234.0:217.0
add:157.0:267.0
add:476.0:354.0
add:148.0:268.0
add:243.0:209.0
add:478.0:351.0
add:130.0:238.0
add:446.0:329.0
add:122.0:254.0
add:235.0:200.0
add:449.0:301.0
add:131.0:240.0
add:237.0:195.0
add:422.0:281.0
add:132.0:236.0
add:245.0:192.0
add:412.0:258.0
add:125.0:227.0
add:272.0:228.0
add:395.0:239.0
add:121.0:219.0
add:231.0:183.0
add:382.0:223.0
add:126.0:231.0
add:237.0:184.0
add:363.0:188.0
add:130.0:229.0
add:248.0:183.0
add:292.0:169.0
add:178.0:256.0
add:135.0:215.0
add:265.0:186.0
add:342.0:143.0
add:58.0:129.0
add:132.0:227.0
add:258.0:225.0
add:139.0:219.0
add:301.0:127.0
add:251.0:157.0
add:139.0:208.0
add:304.0:97.0
add:272.0:158.0
add:142.0:205.0
add:275.0:83.0
add:260.0:165.0
add:139.0:207.0
add:279.0:225.0
add:246.0:75.0
add:137.0:203.0
add:300.0:215.0
add:186.0:203.0
add:286.0:152.0
add:225.0:49.0
add:130.0:212.0
add:216.0:39.0
add:89.0:188.0
add:59.0:132.0
add:335.0:350.0
a

remove:15.0:11.0
remove:12.0:9.0
remove:13.0:12.0
remove:11.0:10.0
remove:12.0:8.0
remove:5.0:13.0
add:31.0:48.0
add:32.0:48.0
remove:9.0:8.0
remove:5.0:13.0
add:28.0:53.0
remove:6.0:15.0
add:27.0:55.0
add:24.0:57.0
remove:5.0:15.0
remove:9.0:7.0
remove:13.0:10.0
add:24.0:28.0
add:27.0:34.0
remove:10.0:8.0
add:35.0:31.0
add:20.0:66.0
remove:13.0:13.0
remove:9.0:7.0
remove:9.0:10.0
remove:14.0:12.0
remove:15.0:12.0
add:54.0:35.0
add:18.0:67.0
remove:12.0:11.0
remove:10.0:10.0
add:5.0:42.0
remove:5.0:9.0
add:56.0:35.0
remove:13.0:10.0
add:9.0:38.0
remove:9.0:10.0
remove:6.0:9.0
add:60.0:37.0
add:11.0:34.0
add:123.0:119.0
remove:5.0:11.0
remove:9.0:8.0
remove:8.0:6.0
add:15.0:80.0
add:68.0:39.0
remove:5.0:10.0
add:113.0:124.0
add:12.0:37.0
remove:7.0:10.0
add:97.0:102.0
add:64.0:37.0
add:14.0:37.0
remove:8.0:6.0
remove:6.0:11.0
add:75.0:39.0
add:91.0:96.0
add:17.0:38.0
add:30.0:18.0
add:39.0:22.0
add:12.0:39.0
add:86.0:83.0
remove:11.0:11.0
add:74.0:87.0
add:48.0:20.0
add:22.0:27.0
add:95

...

add:74.0:152.0
add:154.0:115.0
add:32.0:102.0
add:30.0:173.0
add:35.0:56.0
add:69.0:137.0
add:45.0:140.0
add:89.0:112.0
add:35.0:131.0
add:45.0:103.0
add:198.0:63.0
add:33.0:81.0
add:26.0:111.0
add:39.0:116.0
add:95.0:16.0
add:35.0:93.0
add:30.0:180.0
add:193.0:86.0
add:40.0:66.0
add:40.0:139.0
add:170.0:95.0
add:37.0:78.0
add:68.0:119.0
add:87.0:15.0
add:38.0:76.0
add:69.0:145.0
add:82.0:162.0
add:150.0:102.0
add:34.0:91.0
add:36.0:60.0
add:74.0:160.0
add:41.0:125.0
add:172.0:104.0
add:138.0:155.0
add:36.0:128.0
add:35.0:55.0
add:42.0:68.0
add:44.0:128.0
add:56.0:125.0
add:75.0:158.0
add:34.0:93.0
add:164.0:105.0
add:23.0:143.0
add:81.0:14.0
add:31.0:76.0
add:92.0:92.0
add:46.0:135.0
add:44.0:144.0
add:69.0:135.0
add:36.0:129.0
add:100.0:105.0
add:30.0:181.0
add:58.0:144.0
add:32.0:104.0
add:34.0:129.0
add:70.0:142.0
add:26.0:67.0
add:97.0:108.0
add:106.0:213.0
add:43.0:130.0
add:68.0:132.0
add:91.0:235.0
add:39.0:73.0
add:32.0:98.0
add:31.0:93.0
add:102.0:128.0
add:39.0:74.0
add:35.0

add:47.0:165.0
add:54.0:144.0
add:40.0:162.0
add:63.0:147.0
add:48.0:129.0
add:66.0:95.0
add:37.0:75.0
add:55.0:142.0
add:39.0:172.0
add:54.0:111.0
add:47.0:127.0
add:32.0:96.0
add:63.0:154.0
add:31.0:75.0
add:35.0:54.0
add:30.0:58.0
add:30.0:121.0
add:48.0:137.0
add:41.0:106.0
add:54.0:106.0
add:35.0:77.0
add:35.0:52.0
add:30.0:63.0
add:70.0:160.0
add:62.0:69.0
add:70.0:160.0
add:79.0:15.0
add:26.0:82.0
add:42.0:126.0
add:29.0:109.0
add:35.0:127.0
add:32.0:87.0
add:29.0:67.0
add:72.0:157.0
add:65.0:97.0
add:64.0:62.0
add:52.0:110.0
add:46.0:150.0
add:63.0:151.0
add:36.0:73.0
add:80.0:242.0
add:47.0:121.0
add:36.0:82.0
add:84.0:15.0
add:44.0:175.0
add:45.0:179.0
add:43.0:150.0
add:57.0:143.0
add:62.0:176.0
add:34.0:52.0
add:34.0:72.0
add:82.0:14.0
add:29.0:56.0
add:61.0:70.0
add:39.0:74.0
add:38.0:84.0
add:60.0:127.0
add:47.0:116.0
add:46.0:133.0
add:50.0:109.0
add:37.0:122.0
add:47.0:94.0
add:73.0:124.0
add:50.0:143.0
add:38.0:148.0
add:44.0:109.0
add:52.0:113.0
add:60.0:122.0
add:65.

add:37.0:142.0
add:65.0:202.0
add:40.0:147.0
add:29.0:126.0
add:61.0:167.0
add:32.0:99.0
add:43.0:75.0
add:38.0:67.0
add:52.0:134.0
add:107.0:111.0
add:62.0:161.0
add:43.0:135.0
add:34.0:64.0
add:25.0:76.0
add:27.0:73.0
add:29.0:102.0
add:22.0:65.0
add:33.0:88.0
add:27.0:55.0
add:37.0:73.0
add:46.0:69.0
add:32.0:62.0
add:56.0:93.0
add:35.0:143.0
add:40.0:134.0
add:45.0:138.0
add:67.0:189.0
add:37.0:131.0
add:48.0:154.0
add:112.0:120.0
add:33.0:76.0
add:26.0:74.0
add:30.0:60.0
add:29.0:61.0
add:35.0:81.0
add:27.0:53.0
add:35.0:144.0
add:104.0:114.0
add:28.0:109.0
add:35.0:89.0
add:27.0:102.0
add:31.0:67.0
add:48.0:117.0
add:66.0:170.0
add:42.0:77.0
add:25.0:64.0
add:34.0:96.0
add:20.0:91.0
add:32.0:112.0
add:36.0:147.0
add:59.0:135.0
add:37.0:146.0
add:65.0:186.0
add:29.0:105.0
add:72.0:105.0
add:33.0:77.0
add:51.0:134.0
add:36.0:68.0
add:50.0:90.0
add:25.0:66.0
add:47.0:115.0
add:39.0:63.0
add:50.0:126.0
add:24.0:56.0
add:28.0:63.0
add:41.0:115.0
add:36.0:135.0
add:74.0:148.0
add:27.0:

add:63.0:142.0
add:36.0:135.0
add:40.0:130.0
add:37.0:126.0
add:54.0:116.0
add:50.0:130.0
add:54.0:151.0
add:44.0:135.0
add:111.0:137.0
add:73.0:201.0
add:46.0:134.0
add:75.0:166.0
add:123.0:171.0
add:44.0:138.0
add:39.0:132.0
add:42.0:138.0
add:69.0:178.0
add:50.0:139.0
add:38.0:134.0
add:105.0:140.0
add:41.0:145.0
add:45.0:133.0
add:95.0:155.0
add:65.0:176.0
add:32.0:88.0
add:40.0:109.0
add:60.0:136.0
add:62.0:147.0
add:82.0:118.0
add:47.0:124.0
add:41.0:132.0
add:44.0:149.0
add:63.0:151.0
add:42.0:149.0
add:72.0:108.0
add:43.0:152.0
add:46.0:147.0
add:50.0:157.0
add:42.0:147.0
add:79.0:176.0
add:59.0:89.0
add:51.0:141.0
add:47.0:128.0
add:57.0:143.0
add:48.0:140.0
add:59.0:144.0
add:129.0:181.0
add:52.0:105.0
add:46.0:160.0
add:44.0:126.0
add:42.0:105.0
add:44.0:135.0
add:63.0:146.0
add:48.0:95.0
add:86.0:59.0
add:64.0:110.0
add:57.0:159.0
add:44.0:152.0
add:105.0:122.0
add:60.0:156.0
add:55.0:164.0
add:41.0:132.0
add:48.0:154.0
add:62.0:178.0
add:39.0:114.0
add:75.0:156.0
add:54.0:

add:137.0:100.0
add:91.0:251.0
add:46.0:100.0
add:38.0:64.0
add:50.0:138.0
add:108.0:19.0
add:58.0:44.0
add:59.0:17.0
add:74.0:99.0
add:43.0:102.0
add:50.0:183.0
add:45.0:134.0
add:59.0:169.0
add:38.0:81.0
add:74.0:46.0
add:55.0:109.0
add:52.0:152.0
add:37.0:93.0
add:46.0:167.0
add:92.0:46.0
add:36.0:64.0
add:39.0:87.0
add:44.0:130.0
add:49.0:183.0
add:105.0:39.0
add:105.0:150.0
add:43.0:128.0
add:116.0:34.0
add:185.0:68.0
add:29.0:74.0
add:49.0:99.0
add:51.0:17.0
add:88.0:230.0
add:116.0:154.0
add:152.0:77.0
add:42.0:118.0
add:103.0:210.0
add:30.0:73.0
add:169.0:207.0
add:43.0:153.0
add:32.0:86.0
add:36.0:65.0
add:152.0:90.0
add:69.0:194.0
add:113.0:137.0
add:155.0:177.0
add:83.0:17.0
add:31.0:103.0
add:70.0:208.0
add:38.0:75.0
add:26.0:75.0
add:33.0:72.0
add:45.0:138.0
add:32.0:76.0
add:152.0:105.0
add:84.0:184.0
add:74.0:169.0
add:49.0:108.0
add:46.0:110.0
add:142.0:167.0
add:112.0:17.0
add:126.0:39.0
add:126.0:122.0
add:145.0:94.0
add:65.0:174.0
add:66.0:184.0
add:51.0:119.0
add:13

In [11]:
def get_obj_from_image_file(file, bbox):
    img = cv2.imread(file)
    img_obj = img[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])]
    return img_obj

In [12]:
def generate_new_xmlobj(xmlobj_old, new_position, new_size):
    element_object = ET.Element('object')
    tag_name = ET.SubElement(element_object, 'name')
    tag_name.text = xmlobj_old.find('name').text

    tag_difficult = ET.SubElement(element_object, 'difficult')
    tag_difficult.text = xmlobj_old.find('difficult').text

    element_bndbox = ET.SubElement(element_object, 'bndbox')
    tag_xmin = ET.SubElement(element_bndbox, 'xmin')
    tag_ymin = ET.SubElement(element_bndbox, 'ymin')
    tag_xmax = ET.SubElement(element_bndbox, 'xmax')
    tag_ymax = ET.SubElement(element_bndbox, 'ymax')
    tag_xmin.text = str(new_position[0])
    tag_ymin.text = str(new_position[1])
    tag_xmax.text = str(new_position[0] + new_size[1])
    tag_ymax.text = str(new_position[1] + new_size[0])
    return element_object
    
def past_and_insert(img_obj, img_array, new_position, obj_element, etree):
    new_xmlobj = generate_new_xmlobj(obj_element, new_position, img_obj.shape[:2])
    new_xml_etree = insert_to_xml(new_xmlobj, etree)
    new_pil_img = past_obj_to_background(img_obj, img_array, new_position)
    return new_pil_img, new_xml_etree
    

def insert_to_xml(xml_obj, xml_etree):
    root = xml_etree.getroot()
    root.append(xml_obj)
    return xml_etree
    
def past_obj_to_background(img_obj, img_array, position = (200, 200)):
    img = Image.fromarray(img_array)
    img_obj = Image.fromarray(img_obj)
    img.paste(img_obj, position)
    return img
    
def get_cls_from_xmlobj(obj_element):    
    return obj_element.find('name').text

def get_bbox_from_xmlobj(obj_element):
    xmlbox = obj_element.find('bndbox')
    return [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), 
            float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
def generate_new_position(img_size, img_obj_size):
    array_x = np.arange(int(img_size[1] - img_obj_size[1]))
    array_y = np.arange(int(img_size[0] - img_obj_size[0]))
    random_x = random.sample(list(array_x), 1)[0]
    random_y = random.sample(list(array_y), 1)[0]
    new_position = (random_x, random_y)
    #print(new_position)
    return new_position
    
    
def inset_obj_to_an_image_and_xml(img_obj, img, obj_element, tree):
    new_position = generate_new_position(img.shape[:2], img_obj.shape[:2])
    new_pil_img, new_xml_etree = past_and_insert(img_obj, img, new_position, obj_element, tree)
    
    return new_pil_img, new_xml_etree

In [13]:
gene_xml_names = os.listdir(GENE_XML_DIR)
for xml_name in gene_xml_names:
    '''
    if(not (int(xml_name[:-7]) == 204)):
        continue
    '''
    img_name = GENE_IMG_DIR + xml_name[:-3] + 'jpg'
    xml_name = GENE_XML_DIR + xml_name
    tree=ET.parse(xml_name)
    img = cv2.imread(img_name)
    
    objs = get_obj_from_xml(xml_name)
    i = 0
    for obj_element in objs:
        bbox = get_bbox_from_xmlobj(obj_element)           
        img_obj = get_obj_from_image_file(img_name, bbox)
        cls = get_cls_from_xmlobj(obj_element)
        #print(cls)
        cls_path = Path(GENE_OBJ_DIR + cls)
        if(not cls_path.is_dir()):
            os.system("mkdir " + GENE_OBJ_DIR + cls)
        save_path_name = GENE_OBJ_DIR + cls + "/" + xml_name.split('/')[-1][:-4] + "_" + str(i) + ".jpg"
        #print(save_path_name)
        cv2.imwrite(save_path_name, img_obj)
        i = i+1
        #new_pil_img, new_xml_etree = inset_obj_to_an_image_and_xml(img_obj, img, obj_element, tree)
        #tree = new_xml_etree
        #img = np.array(new_pil_img)

    ## save xml and img
    #new_pil_img.save('generated_' + img_name.split('/')[-1])
    #new_xml_etree.write('generated_' + xml_name.split('/')[-1])
    #break
   

In [ ]:
print(SEED_CHG_DIR)